# Introduction to GMM

## What is this and why am I learning it?
GMM is a way of uncovering structural parameters from data that may or may not be linear. While we have seen over and over again how regressions can uncover unknown "betas" that summarize causal (and correlational) relationships between variables, these "betas" are _linear_: $y$ is a linear function of $\beta$. However, in many of the models we may right down (especially in Trade, Economic Geography, IO...) we will have structural relationships that are not at all linear in the unknown parameters we want to estimate! 

To give a real-world example, take the system of structural equations from the model in Redding and Sturn (2008) (an Economic Geography, Rosen-Roback style model):

$$w_i^{\sigma} = \sum_n L_nw_n (\tau_{ni}/(P_nT_n))^{1-\sigma} = "FMA_i"$$ 

where $P_n$ is the price index,  $w_i$ are wages in city $i$, $L_i$ is labor (population), $\tau_{ni}$ are trade costs from $n$ to $i$, and $T_i$ are unknown productivity parameters. 

Suppose for simplicity that we know $P_n$ (the price index), we have data on $\{w_n, L_n, \tau_{ni}\}_{i,n}$, and we are using a value for $\sigma = 4$ from previous estimates. Then we can use this equation as a GMM moment condition to estimate $T_n$!

## A more simple example

Let's walk through a more stylized example for our purposes. Suppose you have written down an IO model that goes like this:

Firms produce 2 goods $g = 1,2$ using the input $x$ for each. However, each good uses a different technology, like so:

$$y_g = f_g(x)$$

This is essentially what we saw in class. Now to be more explicit, we are going to say that our model assumes Cobb-Douglass technology, so that our production functions look like this:

\begin{align}
y_1 = x_1^{\alpha_1} \\ 
y_2 = x_2^{\alpha_2} 
\end{align}

$x_1, x_2$ index how much of $x$ is used in the production of goods 1, 2 respectively.

Now firms have uncertainty over their prices. Then our first order conditions from the model imply that the following should hold:
\begin{align}
E\left(p_1\alpha_1x_1^{\alpha_1 - 1} - w\right) &= 0 \\ 
E\left(p_2\alpha_2x_2^{\alpha_2-1} - w\right) &= 0 \\
\end{align}

Now, we have data on lots of firms $f = 1,\ldots, N$, and for each firm we observe prices, inputs, and wages. From these data we can use GMM to back out the values of $\alpha_1, \alpha_2$, by applying the analogy principle, which would imply:

\begin{align}
\frac{1}{N}\sum_i \left(p_{1i}\alpha_1x_{1i}^{\alpha_1 - 1} - w_i\right) &= 0 \\ 
\frac{1}{N}\sum_i \left(p_{2i}\alpha_2x_{2i}^{\alpha_2-1} - w_i\right) &= 0 \\
\end{align}

Our steps will be:
1. Simulate this scenario by proposing a DGP.
2. Create data.
3. Set up our moment conditions.
4. Apply the analogy principle to get from expectations to averages.
5. Estimate GMM.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats.distributions as iid

In [ ]:
# Step 1: The DGP
# note that we need 2N
def dgp(true_alphas, N):
    alpha1, alpha2 = true_alphas
    wage = iid.pareto(5).rvs(size=(N,1))
    p_1 = iid.norm(10,2).rvs(size=(N,1))
    p_2 = iid.gamma(3,2).rvs(size=(N,1)) + 10
    
    # create x from these draws, with noise
    x_1 = ... + iid.norm(scale=2).rvs(size=(N,1))
    x_2 = ... + iid.norm(scale=2).rvs(size=(N,1))
    
    return (x_1, x_2, p_1, p_2, wage)

In [ ]:
# Step 2: create data
alpha_true = (0.75, 0.5)
data = dgp(alpha_true, 100)

In [ ]:
data_df = pd.DataFrame(np.c_[data], columns = ["x1", "x2", "p1", "p2", "w"])
data_df.head()

In [ ]:
data_df.describe()

### Creating moment conditions
Our moment conditions will have to be functions of the unknown parameter, since this is... unknown. Here I imitate Ethan's code, with some minor changes. First we set up for each observation _i_, what the moment condition should look like. Since we have 2 moment conditions, I am going to stack these horizontally. What will the dimentions of this be?

In [ ]:
def gj(alphas, data):
    # unpack my inputs
    alpha1, alpha2 = alphas
    x_1, x_2, p_1, p_2, wage = data
    # set up moments (separately)
    moment1 = ...
    moment2 = ...
    # stack the moments next to each other
    moments = np.c_[moment1, moment2]
    
    return moments

In [ ]:
gj(alpha_true, data)[:5] # these are pretty small, bc the alphas are right!

In [ ]:
gj((0.1, 0.2), data)[:5] # these are much larger!

Now, we appply the analogy principle to these moment conditions - by analogy, the sample mean approximate the expectation, so we want to take the mean over N. In our data, the _N_ are running down the _rows_. The rows are the 0-axis in python, so we want to take the mean over ``axis = 0``. If we apply the mean over ``axis = 1``, this would allow us to take the mean across the columns, which wouldn't make sense here.

In [ ]:
def gN(alphas, data):
    # get individual moments
    e = gj(alphas, data)
    # take mean
    gN = e.mean(axis=0).reshape((len(alphas),1))
    return gN

In [ ]:
# test: we want these = 0!
print(f"using true alphas: Eg_j =\n {gN(alpha_true, data)}\n")
print(f"using other alphas: Eg_j =\n {gN((0.9, 0.9), data)}")

Now we have our analogous moment conditions, and GMM will try and find the alphas that minimize the squared errors from these according to:

$$E(g_j(a))'\Omega^{-1} E(g_j(a))$$

(a 2 x 2 matrix).

But... what is $\Omega$? The most efficient estimator would weight by the covariance of the moment conditions. So let's set it up!  

In [ ]:
def invOmega(alphas, data):
    e = gj(alphas, data)
    # recenter
    e = e - e.mean(axis=0)
    N = e.shape[0]
    var = ...
    return np.linalg.inv(var)

In [ ]:
invOmega(alpha_true, data)

In [ ]:
# putting it all together, we get the objective function we want to minimize:
def J(alphas,omega,data):

    g = gN(alphas,data) # Sample moments
    N = data[0].shape[0]

    return (N*...).squeeze() # Scale by sample size

In [ ]:
J(alpha_true,invOmega(alpha_true, data),data)

### The 2-step estimator.
So now we have all the ingredients, we have set up the moment conditions and the objective function to minimize J(). But J() depends on $\Omega$, which is unknown! The idea now will be to proceed in 2 steps:

1. Guess $\Omega$. We'll guess the identity matrix.
2. Minimize J using our guess for $\Omega$.
3. Use our estimated alphas to calculate $\widehat{\Omega}$.
4. Using $\widehat{\Omega}$, re-run GMM to re-estimate the alphas.

In [ ]:
from scipy.optimize import minimize

In [ ]:
# Step 1: Guess Omega.
Omega_guess = np.eye(len(alpha_true))

# Step 2: Minimize J using our guess. 
# We're bringing back our old friend, the minimize function!
minimizer = minimize(lambda a: ..., x0 = [0.5,0.5])
alpha_hat = minimizer.x
print(f"alpha_hat from first stage = {alpha_hat}")

# Step 3: Update Omega
invOmega_hat = invOmega(tuple(alpha_hat), data)
print(f"Omega_hat =\n {invOmega_hat}")

# Step 4: re-run with omega_hat
minimizer2 = minimize(lambda a: ..., x0 = [0.5,0.5])
alpha_hat2 = minimizer2.x
print(f"alpha_hat from second stage = {alpha_hat2}")